In [1]:
from ngsolve import *
from mpi4py import MPI
from netgen.meshing import Mesh
from netgen.occ import unit_square
import meshio
from ngsolve import *
from ngsolve.webgui import Draw
import netgen
from netgen.geom2d import SplineGeometry
from netgen.read_gmsh import ReadGmsh
import time
import numpy as np

In [2]:
mesh_path = "../meshes/mesh_square.msh"
output_path = "../output/interactive/interactive_ngsolve.vtu"

In [3]:
start_time = time.time()

# WARNING!!! If there is an error, it is likely because the version of .msh file is above 2.
msh = ReadGmsh(mesh_path)
msh = Mesh(msh)

print("ElementBoundary=", msh.GetBoundaries())

# Optional GUI
# Draw(msh)

ElementBoundary= ('Left', 'Top', 'Right', 'Bottom')


In [4]:
fes = H1(msh, order=1, dirichlet="Left|Right")

# Trial and test functions
u = fes.TrialFunction()
v = fes.TestFunction()

# Right-hand side
g = sin(6*x)

In [5]:
a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx

f = LinearForm(fes)
f += 10 * g * v * ds("Top")
f += 10 * g * v * ds("Bottom")

# f += 32 * (y*(1-y)+x*(1-x)) * v * dx

pre = preconditioners.Local(a)
a.Assemble()
f.Assemble()

gfu = GridFunction(fes)

In [6]:

inv = CGSolver(a.mat, pre.mat, printrates = True, precision = 1e-10)
# inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
gfu.vec.data = inv*f.vec

end_time = time.time()

print(f"Runtime: {np.round(end_time - start_time, 3)} s")

# Optional GUI
# Draw(gfu)

Runtime: 0.047 s


In [7]:
# vtk = VTKOutput(msh,coefs=[gfu],names=["sol"],filename="output/square/poisson_square_ngsolve",subdivision=0)
# vtk.Do()

In [8]:
import pyvista as pv
sol = gfu.vec.FV().NumPy()

res = pv.read(mesh_path)
res["sol"] = sol

res.save(output_path)